In [52]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
import gensim
from gensim import corpora, models
import string


In [53]:
from nrclex import NRCLex


In [54]:
dataset = pd.read_csv("Suicide_Detection.csv", index_col=[0] )
dataset = dataset.head(1000) # to be removed.. Taking first few rows for now

In [55]:
dataset.head()

,text,class
2,Ex Wife Threatening SuicideRecently I left my ...,suicide
3,Am I weird I don't get affected by compliments...,non-suicide
4,Finally 2020 is almost over... So I can never ...,non-suicide
8,i need helpjust help me im crying so hard,suicide
9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [56]:
dataset["text"] = dataset["text"].str.lower()
dataset.head()

,text,class
2,ex wife threatening suiciderecently i left my ...,suicide
3,am i weird i don't get affected by compliments...,non-suicide
4,finally 2020 is almost over... so i can never ...,non-suicide
8,i need helpjust help me im crying so hard,suicide
9,"i’m so losthello, my name is adam (16) and i’v...",suicide


In [57]:
PUNCT_TO_REMOVE = string.punctuation

def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

dataset["text"] = dataset["text"].apply(lambda text: remove_punctuation(text))
dataset.head()

,text,class
2,ex wife threatening suiciderecently i left my ...,suicide
3,am i weird i dont get affected by compliments ...,non-suicide
4,finally 2020 is almost over so i can never hea...,non-suicide
8,i need helpjust help me im crying so hard,suicide
9,i’m so losthello my name is adam 16 and i’ve b...,suicide


In [58]:
## from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

dataset["text"] = dataset["text"].apply(lambda text: remove_stopwords(text))
dataset.head()

,text,class
2,ex wife threatening suiciderecently left wife ...,suicide
3,weird dont get affected compliments coming som...,non-suicide
4,finally 2020 almost never hear 2020 bad year e...,non-suicide
8,need helpjust help im crying hard,suicide
9,i’m losthello name adam 16 i’ve struggling yea...,suicide


In [59]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

dataset["text"] = dataset["text"].apply(lambda text: lemmatize_words(text))
dataset.head()

,text,class
2,ex wife threatening suiciderecently left wife ...,suicide
3,weird dont get affected compliment coming some...,non-suicide
4,finally 2020 almost never hear 2020 bad year e...,non-suicide
8,need helpjust help im cry hard,suicide
9,i’m losthello name adam 16 i’ve struggling yea...,suicide


In [60]:
dataset['emotions'] = dataset['text'].apply(lambda x: NRCLex(x).affect_frequencies)
dataset.head()

,text,class,emotions
2,ex wife threatening suiciderecently left wife ...,suicide,"{'fear': 0.12307692307692308, 'anger': 0.12307..."
3,weird dont get affected compliment coming some...,non-suicide,"{'fear': 0.06666666666666667, 'anger': 0.0, 'a..."
4,finally 2020 almost never hear 2020 bad year e...,non-suicide,"{'fear': 0.1, 'anger': 0.1, 'anticip': 0.0, 't..."
8,need helpjust help im cry hard,suicide,"{'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't..."
9,i’m losthello name adam 16 i’ve struggling yea...,suicide,"{'fear': 0.14432989690721648, 'anger': 0.10824..."


In [61]:
dataset = pd.concat([dataset.drop(['emotions'], axis = 1), dataset['emotions'].apply(pd.Series)], axis = 1)
dataset.head()

,text,class,fear,anger,anticip,trust,surprise,positive,negative,sadness,disgust,joy,anticipation
2,ex wife threatening suiciderecently left wife ...,suicide,0.123077,0.123077,0.0,0.076923,0.076923,0.076923,0.169231,0.092308,0.076923,0.076923,0.107692
3,weird dont get affected compliment coming some...,non-suicide,0.066667,0.000000,0.0,0.133333,0.133333,0.133333,0.133333,0.000000,0.066667,0.133333,0.200000
4,finally 2020 almost never hear 2020 bad year e...,non-suicide,0.100000,0.100000,0.0,0.150000,0.050000,0.150000,0.100000,0.050000,0.100000,0.100000,0.100000
8,need helpjust help im cry hard,suicide,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,NaN
9,i’m losthello name adam 16 i’ve struggling yea...,suicide,0.144330,0.108247,0.0,0.067010,0.036082,0.077320,0.190722,0.149485,0.061856,0.041237,0.123711


In [62]:
# TO DO
# Limitation: NRCLex can't handle negations
# Figure how negation can be handled